In [15]:
import os
import cv2
import time
import keyboard
import numpy as np
from PIL import ImageGrab, Image
from pynput.keyboard import Key, Controller

kb = Controller()
lst = {0:'w', 1:'a', 2:'s', 3:'d', 4:'nop'}

In [16]:
def move(opt):
    if opt < 4:
        opt = lst[opt]
        hit(opt)

def hit(key):
    kb.press(key)
    time.sleep(0.09)
    kb.release(key)

def roi(img, vertices):
    mask = np.zeros_like(img)
    cv2.fillPoly(mask, vertices, 255)
    masked = cv2.bitwise_and(img, mask)
    return masked

def process_img(original_image):
    processed_img = original_image
    processed_img = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)
    # processed_img = cv2.Canny(processed_img, threshold1=50, threshold2=100)
    vertices = np.array([[440,37],[158,597], [425,664], [1001,665], [1229,247]], np.int32)
    processed_img = roi(processed_img, [vertices])
    return processed_img

In [19]:
ohv = []
instr = []

start = 0
img_num = 0

tmp = np.eye(5, dtype=int)

# %b  %d %H %M
# Jul 18 12:10
t = time.strftime("%b%d_%H%M_", time.localtime())
print('Current time is: ', t[:-1])

for i in range(3):
    print('{}...'.format(3-i))
    time.sleep(1)

# folder = t[:-1]
# os.chdir(os.getcwd() + '\\data')
# os.system('mkdir {}'.format(folder))
# os.chdir('..')

while True:
    screen =  np.array(ImageGrab.grab(bbox=(0,33,1280,699)))
    new_screen = process_img(screen)
    cv2.imshow('monitor', new_screen)
    
    p = 4
    for i in range(4):
        if keyboard.is_pressed(lst[i]): 
            p = i
            if not start:
                start = 1
            print('img {} - {}'.format(img_num, lst[i]))
            instr.append(i)
            time.sleep(0.1)
            
    if p == 4:
        # print('img {} - NOP'.format(img_num))
        instr.append(4)
    
    img_id = t + str(img_num)
    Image.fromarray(new_screen).save('data/{}/{}.png'.format(lst[p], img_id))
    img_num += 1
    if cv2.waitKey(25) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break

Current time is:  Jul19_1541
3...
2...
1...
img 61 - w
img 65 - w
img 70 - w
img 76 - w
img 91 - w
img 94 - w
img 100 - w
img 120 - w
img 123 - w
img 133 - w
img 137 - w
img 144 - w
img 147 - w
img 161 - w
img 165 - w
img 171 - w


In [8]:
for i in range(5):
    print(lst[i] + '\t' + str(instr.count(i)))

w	69
a	1
s	0
d	3
NOP	356


In [10]:
print('-----\t[w a s d \\]')

for idx,ele in enumerate(instr):
    
    ele = tmp[ele]
    ohv.append(ele)
    
    print('img {}\t'.format(idx) + str(ele))

-----	[w a s d \]
img 0	[0 0 0 0 1]
img 1	[0 0 0 0 1]
img 2	[0 0 0 0 1]
img 3	[0 0 0 0 1]
img 4	[0 0 0 0 1]
img 5	[0 0 0 0 1]
img 6	[0 0 0 0 1]
img 7	[0 0 0 0 1]
img 8	[0 0 0 0 1]
img 9	[0 0 0 0 1]
img 10	[0 0 0 0 1]
img 11	[0 0 0 0 1]
img 12	[0 0 0 0 1]
img 13	[0 0 0 0 1]
img 14	[0 0 0 0 1]
img 15	[0 0 0 0 1]
img 16	[0 0 0 0 1]
img 17	[0 0 0 0 1]
img 18	[0 0 0 0 1]
img 19	[0 0 0 0 1]
img 20	[0 0 0 0 1]
img 21	[0 0 0 0 1]
img 22	[0 0 0 0 1]
img 23	[0 0 0 0 1]
img 24	[0 0 0 0 1]
img 25	[0 0 0 0 1]
img 26	[0 0 0 0 1]
img 27	[0 0 0 0 1]
img 28	[0 0 0 0 1]
img 29	[0 0 0 0 1]
img 30	[0 0 0 0 1]
img 31	[0 0 0 0 1]
img 32	[0 0 0 0 1]
img 33	[0 0 0 0 1]
img 34	[0 0 0 0 1]
img 35	[0 0 0 0 1]
img 36	[0 0 0 0 1]
img 37	[0 0 0 0 1]
img 38	[0 0 0 0 1]
img 39	[0 0 0 0 1]
img 40	[0 0 0 0 1]
img 41	[0 0 0 0 1]
img 42	[0 0 0 0 1]
img 43	[0 0 0 0 1]
img 44	[0 0 0 0 1]
img 45	[0 0 0 0 1]
img 46	[0 0 0 0 1]
img 47	[0 0 0 0 1]
img 48	[0 0 0 0 1]
img 49	[0 0 0 0 1]
img 50	[0 0 0 0 1]
img 51	[0 0 0 0 1]
img 